In [ ]:
!pip install h5sparse
!pip install tensorflow==2.1.1
!pip install numpy==1.18.0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import numpy as np
from scipy import sparse
import os
from sklearn import tree
import h5sparse
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding


In [ ]:
def sample_timesteps(X: np.ndarray):
    X_with_100_timesteps = []
    for i, replay in enumerate(X):
        timesteps = X[i].shape[0]
        timesteps_per_percent = int(timesteps / 100)
        X_with_100_timesteps.append(np.array(replay[0::timesteps_per_percent][:]))
    X = np.array(X_with_100_timesteps)
    return X

In [ ]:
# stolen from https://stackoverflow.com/questions/47200146/keras-load-images-batch-wise-for-large-dataset

test_set_d = "/content/gdrive/My Drive/University/bachelors project/bachelors-thesis/thesis/0.npz"

def load_test_set(d):
    # load in replays from disk
    npzfile = np.load(d, allow_pickle=True)
    replays = npzfile["arr_0"]

    # split dataset into X and Y
    Y = []
    X = []
    for replay in replays:
        Y.append(replay[0, 0])
        X.append(replay[:, 1:])
    X = np.array(X)
    Y = np.array(Y)

    X = sample_timesteps(X)
    X = tf.keras.preprocessing.sequence.pad_sequences(X, padding="post")

    return X, Y


def load_replays(start, end, hf):

    batch = []
    for i in range(start, end):
        batch.append(hf[f"sparse/{i}"][:].toarray())

    Y = []
    #for replay in batch:
    #    Y.append(replay[0, 0])
    #batch = tf.keras.preprocessing.sequence.pad_sequences(batch, padding="pre", dtype='float')
    # split dataset into X and Y
    X = []
    for replay in batch:
        X.append(replay[-101:, 1:])
        Y.append(replay[0, 0])
    X = np.array(X)
    Y = np.array(Y)

    # X = sample_timesteps(X)

    return X, Y


def replay_loader(batch_size: int, training_data):
    L = len(training_data["sparse"])

    # this line is just to make the generator infinite, keras needs that
    while True:
        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            X, Y = load_replays(batch_start, limit, training_data)

            yield (X, Y)  # a tuple with two numpy arrays with batch_size samples

            batch_start += batch_size
            batch_end += batch_size

In [ ]:
def load_test_replays(start, end, hf):

    batch = []
    for i in range(start, end):
        batch.append(hf[f"sparse/{i}"][:].toarray())
    
    Y = []
    #for replay in batch:
    #    Y.append(replay[0, 0])
    #batch = tf.keras.preprocessing.sequence.pad_sequences(batch, padding="pre", dtype='float')
    # split dataset into X and Y
    X = []
    for replay in batch:
        Y.append(replay[0, 0])
        X.append(replay[-101:, 1:])
    X = np.array(X)
    Y = np.array(Y)

    # X = sample_timesteps(X)

    return X, Y

test_set_length = 979
test_h5 = h5sparse.File("/content/gdrive/My Drive/University/bachelors project/bachelors-thesis/thesis/test.h5", "r") # test data
X_test, Y_test = load_test_replays(0, test_set_length, test_h5)

# X_test = sample_timesteps(X_test)
# X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding="post")

In [ ]:
print(Y_test.shape)
print(X_test[20].shape)

(979,)
(101, 752)


In [ ]:
# x_test, y_test = load_test_set(test_set_d)

In [ ]:
#, input_shape=X_test[0].shape

simple_lstm_model = tf.keras.models.Sequential(
    [tf.keras.layers.LSTM(128, input_shape=X_test[0].shape, return_sequences=True),
    #  tf.keras.layers.Dropout(0.2),
    #  tf.keras.layers.LSTM(32, return_sequences=True),
    #  tf.keras.layers.Dropout(0.5),
    #  tf.keras.layers.LSTM(32),
     tf.keras.layers.Dense(units=1),
    #  tf.keras.layers.Activation('sigmoid'),
     ],
)

simple_lstm_model.compile(optimizer="adam", loss="mae")

In [ ]:
d = "/content/gdrive/My Drive/University/bachelors project/bachelors-thesis/thesis/train.h5"
hf = h5sparse.File(d, "r") # training data

In [ ]:
batch_size = 16
epochs_lll=489
simple_lstm_model.fit(replay_loader(batch_size, hf), epochs=489, validation_data=(X_test, Y_test), steps_per_epoch=batch_size)

  ...
    to  
  ['...']
Train for 16 steps, validate on 979 samples
Epoch 1/489
16/16 [==============================] - 5s 343ms/step - loss: 0.4755 - val_loss: 0.4745
Epoch 2/489
16/16 [==============================] - 3s 202ms/step - loss: 0.4869 - val_loss: 0.4671
Epoch 3/489
16/16 [==============================] - 3s 202ms/step - loss: 0.4599 - val_loss: 0.4482
Epoch 4/489
16/16 [==============================] - 3s 200ms/step - loss: 0.4472 - val_loss: 0.4626
Epoch 5/489
16/16 [==============================] - 3s 200ms/step - loss: 0.4468 - val_loss: 0.4389
Epoch 6/489
16/16 [==============================] - 3s 200ms/step - loss: 0.4107 - val_loss: 0.4439
Epoch 7/489
16/16 [==============================] - 3s 200ms/step - loss: 0.4356 - val_loss: 0.4224
Epoch 8/489
16/16 [==============================] - 3s 201ms/step - loss: 0.4147 - val_loss: 0.4210
Epoch 9/489
16/16 [==============================] - 3s 204ms/step - loss: 0.4261 - val_loss: 0.3969
Epoch 10/489
16/16 [==

In [ ]:
model_path = "/content/gdrive/My Drive/University/bachelors project/bachelors-thesis/thesis/128_x1_lstm.savedmodel/"
simple_lstm_model.save(model_path)

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/University/bachelors project/bachelors-thesis/thesis/64_x1_lstm.savedmodel/assets


In [ ]:
 # This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))